In [9]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Read file "hw1_data.tsv"
file = open("/content/drive/MyDrive/hw1_data.tsv",'r')
ori_data = file.read().strip().split("\n")
data = []
for item in ori_data:
  item = item.split("\t")
  if len(item[0]) >0:
    data.append((item[0],item[1]))
print(len(data))

300


In [11]:
### Training data pre-processing
texts, labels = [], []
label2idx = {"0":0, "1":1}
for i, item in enumerate(data):
  text = item[0]

  ## Preprocessing (if you want to add, please add more)
  ################################################################################
  text = text.replace("  "," ") ## Replace double space
  text = text.replace(",", "") ## Replace comma to ""
  text = text.lower()  ## Lower cases
  ################################################################################
  label = label2idx[item[1]]

  texts.append(text)
  labels.append(label)

print("*"*50)
print("Total number of datasets")
print(len(texts))
print(len(labels))
print("*"*50)




**************************************************
Total number of datasets
300
300
**************************************************


In [12]:
#### Split into train/dev/test sets
from sklearn.model_selection import train_test_split
### Write a code for collecting samples for each class
################################################################################
pos,neg = [], []
train_texts, dev_texts, test_texts, train_labels, dev_labels, test_labels = [], [], [], [], [], []
for a,b in zip(texts,labels):
  if b == 1:
    pos.append((a,b))
  else:
    neg.append((a,b))

for a,b in pos[:50]:
  test_texts.append(a)
  test_labels.append(b)
for a,b in neg[:50]:
  test_texts.append(a)
  test_labels.append(b)

for a,b in pos[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)
for a,b in neg[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)

for a,b in pos[100:]:
  train_texts.append(a)
  train_labels.append(b)
for a,b in neg[100:]:
  train_texts.append(a)
  train_labels.append(b)
################################################################################


tmp = list(zip(train_texts,train_labels))
import random
random.shuffle(tmp)
train_texts, train_labels = zip(*tmp)

tmp = list(zip(dev_texts,dev_labels))
import random
random.shuffle(tmp)
dev_texts, dev_labels = zip(*tmp)

tmp = list(zip(test_texts,test_labels))
import random
random.shuffle(tmp)
test_texts, test_labels = zip(*tmp)

print("Train Dataset Examples")
print(train_texts[:3])
print(train_labels[:3])
print("*"*50)
print(train_labels)
print(dev_labels)

Train Dataset Examples
("it 's smooth and professional ", 'an admittedly middling film ', 'takes hold and grips hard ')
(1, 1, 1)
**************************************************
(1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0)
(0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0)


In [13]:
## Construct a vocabulary
from collections import Counter

all_words = []
for item in train_texts:
  all_words += item.split()
for item in dev_texts:
  all_words += item.split()

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {'<pad>':0, "<unk>":1}
vocab_to_int.update({word: ii for ii, word in enumerate(vocab,2)})
print(vocab_to_int)
print(len(vocab_to_int))

{'<pad>': 0, '<unk>': 1, 'the': 2, 'and': 3, 'a': 4, 'to': 5, '.': 6, 'of': 7, 'it': 8, 'in': 9, 'that': 10, 'is': 11, "'s": 12, 'as': 13, 'for': 14, 'film': 15, 'you': 16, 'with': 17, 'one': 18, 'movie': 19, 'by': 20, 'this': 21, 'its': 22, '--': 23, '...': 24, 'have': 25, 'so': 26, 'story': 27, 'good': 28, 'most': 29, 'more': 30, 'not': 31, 'your': 32, "n't": 33, 'no': 34, 'time': 35, 'if': 36, 'an': 37, 'takes': 38, 'little': 39, 'on': 40, 'be': 41, 'but': 42, 'i': 43, 'too': 44, 'about': 45, 'way': 46, 'are': 47, '(': 48, ')': 49, 'own': 50, 'very': 51, 'some': 52, 'can': 53, '``': 54, 'from': 55, 'could': 56, 'worst': 57, 'when': 58, 'picture': 59, 'at': 60, 'just': 61, 'or': 62, 'his': 63, 'comedy': 64, 'life': 65, 'surprisingly': 66, 'what': 67, 'like': 68, 'watching': 69, 'old': 70, 'better': 71, 'their': 72, 'every': 73, 'all': 74, 'does': 75, 'hate': 76, "''": 77, 'many': 78, 'recent': 79, 'into': 80, 'moments': 81, 'completely': 82, 'work': 83, 'men': 84, 'seems': 85, 'despi

In [14]:
import numpy as np
def encode_sentence(sentence):
    max_length = 50
    input_ids = []
    for item in sentence.split():
      if item in vocab_to_int:
        input_ids.append(vocab_to_int[item])
      else:
        input_ids.append(vocab_to_int['<unk>'])

    padding_length = max_length - len(input_ids)
    input_ids += [vocab_to_int['<pad>']] * padding_length
    return np.array(input_ids)

def encode_label(label):
    return np.array(label)



print("Train Dataset Encode Examples")
for a,b in zip(train_texts[:3],train_labels[:3]):
  print(encode_sentence(a),b)

Train Dataset Encode Examples
[  8  12 195   3 196   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1
[ 37 197 198  15   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1
[ 38 199   3 200 201   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1


In [15]:
import numpy as np
from sklearn.metrics import accuracy_score

class SimpleNN:
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        # Initialize weights and biases
        self.embedding_weights = np.random.rand(vocab_size, embedding_dim) # Embedding weights
        limit = np.sqrt(6 / (embedding_dim + hidden_size))
        self.W1 = np.random.uniform(-limit, limit, size=(embedding_dim, hidden_size))
        limit = np.sqrt(6/ (hidden_size+output_size))
        self.W2 = np.random.uniform(-limit, limit, size=(hidden_size, output_size))
        # self.W1 = np.random.rand(embedding_dim, hidden_size)   # Input to hidden weights
        # self.W2 = np.random.rand(hidden_size, output_size) # Hidden to output weights
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.b2 = np.zeros((1, output_size))  # Output layer biases
        self.X = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)

    def forward(self, X):
        # Forward pass
        embedded_x = self.embedding_weights[X]
        self.sentence_vec = np.mean(embedded_x, axis=0, keepdims=True)

        z1 = np.dot(self.sentence_vec, self.W1) + self.b1
        self.h = self.sigmoid(z1)

        z2 = np.dot(self.h, self.W2) + self.b2
        output = self.sigmoid(z2)
        return output

    def compute_loss(self, y, output):
        # Compute binary cross-entropy loss
        return -np.sum(y * np.log(output) + (1 - y) * np.log(1 - output))

    def backward(self, X, y, output, learning_rate=0.01):
      delta2 = output - y
      grad_W2 = np.dot(self.h.T, delta2)
      grad_b2 = np.sum(delta2, axis=0, keepdims=True)

      delta1 = np.dot(delta2, self.W2.T) * self.sigmoid_derivative(self.h)
      grad_W1 = np.dot(self.sentence_vec.T, delta1)
      grad_b1 = np.sum(delta1, axis=0, keepdims=True)

      self.W1 -= learning_rate * grad_W1
      self.b1 -= learning_rate * grad_b1
      self.W2 -= learning_rate * grad_W2
      self.b2 -= learning_rate * grad_b2

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def train(X, y, learning_rate=0.01):
    output = nn.forward(X)  # Forward pass
    loss = nn.compute_loss(y, output)  # Compute loss
    nn.backward(X, y, output, learning_rate)  # Backward pass
    return loss

def predict(x):
    output = nn.forward(x)
    return output, (output > 0.5).astype(int)  # Binary classification

# Initialize the neural network
vocab_size = len(vocab_to_int)  # Number of unique words in vocab
embedding_dim = 100  # Embedding dimension
hidden_size = 80  # Number of neurons in the hidden layer
output_size = 1  # One output (binary classification)
learning_rate = 0.01
nn = SimpleNN(vocab_size, embedding_dim, hidden_size, output_size)

true, pred = [], []
epochs = 100
for epoch in range(epochs):
    train_loss = 0.0
    true, pred = [], []  # Reset true and predicted labels for each epoch
    for x, y in zip(train_texts, train_labels):
        x = encode_sentence(x)  # Encode sentence as word indices
        train_loss += train(x, y, learning_rate)  # Train on the current sample
        _, prediction = predict(x)  # obtain prediction
        true.append(y)  # Append true label
        pred.append(prediction[0][0])  # Append predicted label (extract scalar)

    # Calculate training accuracy
    train_acc = accuracy_score(true, pred) * 100.0
    train_loss /= len(train_texts)  # Average training loss

    # Evaluate on dev set
    dev_true, dev_pred = [], []
    dev_loss = 0.0
    for x, y in zip(dev_texts, dev_labels):
        x = encode_sentence(x)

        output, prediction = predict(x)
        dev_loss += nn.compute_loss(y,output)
        dev_true.append(y)
        dev_pred.append(prediction[0][0])
    dev_loss /= len(dev_texts)
    dev_acc = accuracy_score(dev_true, dev_pred) * 100.0
    print(f'{epoch} epoch, train_loss = {train_loss:.4f}, train_acc: {train_acc:.2f}%, eval_loss: {dev_loss:.4f}, eval_acc: {dev_acc:.2f}%')


0 epoch, train_loss = 0.7216, train_acc: 69.00%, eval_loss: 0.6941, eval_acc: 50.00%
1 epoch, train_loss = 0.7205, train_acc: 66.00%, eval_loss: 0.6942, eval_acc: 50.00%
2 epoch, train_loss = 0.7203, train_acc: 66.00%, eval_loss: 0.6942, eval_acc: 50.00%
3 epoch, train_loss = 0.7202, train_acc: 66.00%, eval_loss: 0.6943, eval_acc: 50.00%
4 epoch, train_loss = 0.7201, train_acc: 66.00%, eval_loss: 0.6943, eval_acc: 50.00%
5 epoch, train_loss = 0.7199, train_acc: 66.00%, eval_loss: 0.6944, eval_acc: 50.00%
6 epoch, train_loss = 0.7198, train_acc: 66.00%, eval_loss: 0.6944, eval_acc: 50.00%
7 epoch, train_loss = 0.7196, train_acc: 66.00%, eval_loss: 0.6944, eval_acc: 50.00%
8 epoch, train_loss = 0.7195, train_acc: 66.00%, eval_loss: 0.6945, eval_acc: 50.00%
9 epoch, train_loss = 0.7193, train_acc: 66.00%, eval_loss: 0.6945, eval_acc: 50.00%
10 epoch, train_loss = 0.7192, train_acc: 66.00%, eval_loss: 0.6946, eval_acc: 50.00%
11 epoch, train_loss = 0.7191, train_acc: 66.00%, eval_loss: 0.6